In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
mnist_dataset ,mnist_info = tfds.load(name ='mnist', with_info=True, as_supervised =True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /home/pranav/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [3]:
mnist_train ,mnist_test = mnist_dataset['train'] , mnist_dataset['test']

num_validation_samples = .1 * mnist_info.splits['train'].num_examples # could be int or not
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples ,tf.int64)



In [4]:
def scale(image ,label):
    image = tf.cast(image ,tf.float32)
    image /= 255  #to converting val bw 0 and 1 
    return image , label
scaled_train_validate_data = mnist_train.map(scale)  #data_set.map(fn) -> fn(data_set)
test_data = mnist_test.map(scale)

In [6]:
BUFFER_SIZE  =10000   #shuffling 10000 samples at once

In [7]:
shuffled_train_validate_data = scaled_train_validate_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_validate_data.take(num_validation_samples)
train_data = shuffled_train_validate_data.skip(num_validation_samples)

#Batching gradient descent
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data =validation_data.batch(num_validation_samples)  # single batch of total validaton set
test_data =test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# outline model


In [17]:
input_size = 784  #first layer
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape =(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                        ])

# CHoosing optimizer and loss function

In [18]:
model.compile(optimizer ='adam' ,loss ='sparse_categorical_crossentropy', metrics =['accuracy'])

# TRAINING

In [19]:
NUM_EPOCHs = 5

model.fit(train_data, epochs = NUM_EPOCHs,validation_data = (validation_inputs,validation_targets),validation_steps=10, verbose =2)

Epoch 1/5
540/540 - 13s - loss: 0.3309 - accuracy: 0.9052 - val_loss: 0.1612 - val_accuracy: 0.9562
Epoch 2/5
540/540 - 12s - loss: 0.1340 - accuracy: 0.9600 - val_loss: 0.1074 - val_accuracy: 0.9693
Epoch 3/5
540/540 - 12s - loss: 0.0938 - accuracy: 0.9717 - val_loss: 0.0815 - val_accuracy: 0.9740
Epoch 4/5
540/540 - 12s - loss: 0.0739 - accuracy: 0.9777 - val_loss: 0.0767 - val_accuracy: 0.9775
Epoch 5/5
540/540 - 12s - loss: 0.0585 - accuracy: 0.9816 - val_loss: 0.0612 - val_accuracy: 0.9815


In [20]:
test_loss, test_accuracy = model.evaluate(test_data)
test_loss, test_accuracy

      1/Unknown - 2s 2s/step - loss: 0.0804 - accuracy: 0.9754

(0.08042366802692413, 0.9754)